# P0 with Exception Test Cases

#### Author: Kevin Zhou, Fanping Jiang, Meijing Li

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from IPython.display import display
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

    
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Scanner Tests for Exception

In [ ]:
import SC
def scanString(src):
    SC.init(src); syms = []
    while SC.sym != SC.EOF:
        syms.append(('INDENT' if SC.sym == SC.INDENT else \
                     'DEDENT' if SC.sym == SC.DEDENT else \
                     'IDENT' if SC.sym == SC.IDENT else SC.sym, SC.newline))
        SC.getSym()
    return syms

### Parsing Explicit Exceptions

In [ ]:
scanString("""

program p

  if a then
    throw 
  else
    throw
  if a then writeln() else writeln()
""")

In [ ]:
scanString("""

program p

  if a then
    throw 66
  else
    throw 88
  if a then throw 99 else writeln()
""")

### Tests for Explicit Exceptions

### Square Root Calculator

- Calculate the square root of `-2`

Since `-2` is smaller than 0, `throw $e28` is generated and outputs `123` as expected.

Run the cell below to see the generated WebAssembly code.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 28
        write(123)
    catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 28
        write(123)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm('test.wasm')

- Since the exception can be any positive integers, you can have whatever positive integers you would like it to be.

Now we change the `throw 28` to `throw 42` as well as `catch 28` to `catch 42`, and run the cells below to see the result.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 42 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 42
        write(123)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm('test.wasm')

- You have to match the statement in order to catch correctly, otherwise it will not catch as expected.

Now we change the `catch 42` to `catch 20`, and run the cells below to see the result.

Since there is no `statement 20`, it will print nothing.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 42 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(-2)
        write(a)
    catch 20
        write(123)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm('test.wasm')

- Finally, we calculate the square root of `81`

Since `81` is greater than 0, it outputs the square root of `81` which is `9` as expected.

Run the cell below to see the generated WebAssembly code.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(81)
        write(a)
    catch 28
        write(123)
    catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [ ]:
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else 
        r := 1
        while (r × r) ≤ x do
            r := r + 1
        r := r - 1
program equationsolver
    var a: integer
    try
        a ← sqrt(81)
        write(a)
    catch 28
        write(123)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm('test.wasm')

### Rounding Up Numbers

The test cases below is for showing the ability of multiple `catch` application.

It prints `1` if it is a negative integer and prints `10` if it is a positive integer. It prints out `0` while the input number is `0`.



<br>

Run the cell below to see the generated WebAssembly code.

In [ ]:
compileString("""
procedure posOrNeg(x: integer) → (r: integer)
    if x < 0 then throw 65536
    if x > 0 then throw 65535
    else r := x 
program positiveOrNegative
    var x, a: integer
    try
        x ← read()
        a ← posOrNeg(x)
        write(a)
    catch 65536
        write(1)
    catch 65535
        write(10)
    catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [ ]:
compileString("""
procedure posOrNeg(x: integer) → (r: integer)
    if x < 0 then throw 65536
    if x > 0 then throw 65535
    else r := x 
program positiveOrNegative
    var x, a: integer
    try
        x ← read()
        a ← posOrNeg(x)
        write(a)
    catch 65536
        write(1)
    catch 65535
        write(10)
    catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm('test.wasm')

## Implicit Exception Test Cases

### Zero Division and Mod by Zero Exception

The zero division exception is built-in with the label of `111`, the below `P0` code is trying to calculate `99/(34*5-170)`, which is `99/0`.


Run the cell below to see the generated WebAssembly code.

In [ ]:
# logic: if a == 0 then throw else...
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 div a)
            write(a)
        catch 111
        write(111)
        catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [ ]:
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 div a)
            write(a)
        catch 111
        write(111)
        catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")

Now we trying to calculate `99/(34*5-170+3)`, which is `99/3`, and it should output `3` instead of raising exception.


Run the cell below to see the generated WebAssembly code.

In [ ]:
# logic: if a == 0 then throw else...
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170 + 3)
            b := (99 div a)
            write(a)
        catch 111
        write(111)
        catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [ ]:
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170 + 3)
            b := (99 div a)
            write(a)
        catch 111
        write(111)
        catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")

Now we trying to calculate the `99/(34*5-170+3)`, which is `99/3`, and it should output `3` instead of raising exception.


Run the cell below to see the generated WebAssembly code.

In [ ]:
# logic: if a == 0 then throw else...
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170 + 3)
            b := (99 div a)
            write(a)
        catch 111
        write(111)
        catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [115]:
compileString("""
program DividedByZero
    var a, b: integer
        try
            a := (34 × 5 - 170 + 3)
            b := (99 div a)
            write(a)
        catch 111
        write(111)
        catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")

- The same story goes under the `Mod by 0` exception, with the difference of the exception is labelled as `112` instead of `111`.

The code below is trying to get the remainder of `99/(34*5-170)`, which is `99/0`, it should raise the `Mod by 0` exception and output `112`.


Run the cell below to see the generated WebAssembly code.

In [ ]:
# logic: if a == 0 then throw else...
compileString("""
program ModByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 mod a)
            write(a)
        catch 112
        write(112)
        catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [ ]:
compileString("""
program ModByZero
    var a, b: integer
        try
            a := (34 × 5 - 170)
            b := (99 mod a)
            write(a)
        catch 112
        write(112)
        catchall
""", 'test.wat')

In [ ]:
!wat2wasm --enable-exceptions test.wat

In [ ]:
runwasm("test.wasm")

Now we trying to calculate the remainder of `99/(34*5-170+2)`, which is `99/2`, and it should output `2` instead of raising exception.


Run the cell below to see the generated WebAssembly code.

In [ ]:
# logic: if a == 0 then throw else...
compileString("""
program ModByZero
    var a, b: integer
        try
            a := (34 × 5 - 170 + 2)
            b := (99 mod a)
            write(a)
        catch 112
        write(112)
        catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [116]:
compileString("""
program ModByZero
    var a, b: integer
        try
            a := (34 × 5 - 170 + 2)
            b := (99 mod a)
            write(a)
        catch 112
        write(112)
        catchall
""", 'test.wat')

In [117]:
!wat2wasm --enable-exceptions test.wat

In [118]:
runwasm("test.wasm")

<IPython.core.display.Javascript object>

### Index Out of Bounds Exception

The `index out of bounds` exception is built-in with the label of `110`. Below is an example.


Run the cell below to see the generated WebAssembly code.

In [ ]:
# logic: if (i on stack < a.tp.lower) or (i on stack >= a.tp.lower + a.tp.length) then throw else ...
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    try
        i := 5
        write(x[i])
    catch 110
        write(110)
    catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [120]:
# for testing the WebAssembly code execution
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    try
        i := 5
        write(x[i])
    catch 110
        write(110)
    catchall
""", 'test.wat')

In [121]:
!wat2wasm --enable-exceptions test.wat

In [122]:
runwasm("test.wasm")

<IPython.core.display.Javascript object>

- What if the index is negative?


Run the cell below to see the generated WebAssembly code.

In [ ]:
# logic: if (i on stack < a.tp.lower) or (i on stack >= a.tp.lower + a.tp.length) then throw else ...
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    try
        i := -5
        write(x[i])
    catch 110
        write(110)
    catchall
""")

Then run the cell below to execute the generated WebAssembly code.

In [124]:
# for testing the WebAssembly code execution
compileString("""
var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    try
        i := -5
        write(x[i])
    catch 110
        write(110)
    catchall
""", 'test.wat')

In [125]:
!wat2wasm --enable-exceptions test.wat

In [126]:
runwasm("test.wasm")

<IPython.core.display.Javascript object>